In [1]:
import os
os.chdir('..')

In [ ]:
import polars as pl

from config import BINANCE_DATA_DIR, TradeType
from generate.resample import polars_calc_resample
from util.time import convert_interval_to_timedelta
from util.ts_manager import TSManager

pl.Config.set_tbl_rows(30)

In [ ]:
um_futures_1m_kline_dir = BINANCE_DATA_DIR / "results_data" / "spot" / "klines" / "1m"
print(um_futures_1m_kline_dir)

symbol = 'BTCUSDT'
df_btc_1m = pl.read_parquet(um_futures_1m_kline_dir / f'{symbol}.pqt')
display(df_btc_1m.tail(5))

df_btc_1h = polars_calc_resample(df_btc_1m, '1m', '1h', '0m')
display(df_btc_1h.tail(5))

In [ ]:
um_futures_1m_kline_dir = BINANCE_DATA_DIR / "results_data" / "um_futures" / "klines" / "1m"
print(um_futures_1m_kline_dir)

symbol = 'BTCUSDT'
df_btc_1m = pl.read_parquet(um_futures_1m_kline_dir / f'{symbol}.pqt')
display(df_btc_1m.tail(5))

df_btc_1h = polars_calc_resample(df_btc_1m, '1m', '1h', '0m')
display(df_btc_1h.tail(10))

In [ ]:
resampled_kline(TradeType.spot, "BTCUSDT", "1h", "5m")

df_btc_1h = pl.read_parquet(
    "/home/lostleaf/crypto_data/binance_data/results_data/spot/resampled_klines/1h/35m/BTCUSDT.pqt"
)

df_btc_1h.tail()